### Exemple de workflow Nicolas Rochet avec dataset Spotify

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns

# Dans ce problème, vous allez réaliser un clustering d'un jeu de données de chansons provenant du site de spotify afin de créer des **clusters de chansons** ayant des propriétés similaires.  
# Optionnellement, vous pourrez tenter de vous servir de votre meilleur modèle pour créer un **système de recommandation** capable d'assigner une chanson non traitée aux 5 chansons les plus proches.

# train data set
train_data = pd.read_csv("https://filedn.eu/lefeldrXcsSFgCcgc48eaLY/datasets/clustering/spotify_dataset_full.csv")
# This data set don't have a test set. Don't forget to make one from training set !


## 0 - Splitter les données avec la methode hold out
#y=train_data["popularity"]
#X=train_data.drop(columns=["popularity"])
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=42)

X_train=train_data.drop(columns=["popularity"])
y=train_data["popularity"]


In [2]:
#Drop data inutiles

X_train=X_train.drop(columns=["track"])
X_train=X_train.drop(columns=["uri"])
X_train=X_train.drop(columns=["artist"])


## Preprocessing steps

In [3]:
# encodage des données catégorielles (decade) avec OrdinalEncoder

from sklearn.preprocessing import OrdinalEncoder
#valeurs_uniques = X_train['decade'].unique()

#Sélection de la colonne à encoder
column_to_encode = X_train[['decade']]
#Encodage de la colonne avec OrdinalEncoder en spécifiant l'ordre
order = [['60s','70s','80s','90s','00s','10s']]  # Spécifier l'ordre des catégories
enc = OrdinalEncoder(categories=order)
encoded_column = enc.fit_transform(column_to_encode)
#Ajout de la colonne encodée au DataFrame original
X_train['decade_encoded'] = encoded_column
X_train=X_train.drop(columns=["decade"])


# scaling : 
from sklearn.preprocessing import RobustScaler
X_train_transf = RobustScaler().fit(X_train)
X_train_transf=X_train_transf.transform(X_train)


## Now let's do some deep learning !

In [4]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, optimizers
# Load the TensorBoard notebook extension
%load_ext tensorboard

# transforme le data set X_train, y_train en tenseur
#X_train = tf.data.Dataset.from_tensors(X_train_transf)
#y_train = tf.data.Dataset.from_tensors(y_train)



train_dataset = tf.data.Dataset.from_tensor_slices((X_train_transf,y))
# Optionnel: Appliquer des transformations comme le batching
#train_dataset = train_dataset.batch(32)


In [5]:
## si vous avez des couches de pre-traitement
#normalizer = tf.keras.layers.Normalization(axis=-1)
#normalizer.adapt(tensor_dataset)
#X_train = normalized(X_train)
#y_train = normalized(y_train)

In [7]:

# construit le modèle (ici avec des couches denses)
model = tf.keras.models.Sequential()
model.add(layers.Dense(100, activation="relu",input_shape=(16,)))
#model.add(layers.Dense(100, activation="relu"))
#model.add(layers.Dense(100, activation="relu"))
#model.add(layers.Dense(100, activation="relu", kernel_regularizer=regularizers.L1(0.03)))
#model.add(layers.Dense(100, activation='relu', bias_regularizer=regularizers.L2(0.01)))
#model.add(layers.Dense(100, activation="relu", bias_regularizer='l2'))
#...

# [Optionnel] applique des couches spécifiques (Normalisation, Drop out, Batch Normalisation)
#model.add(layers.Dropout(rate=0.2))
#model.add(BatchNormalization())

# régularisation sur le poids des neurones
#reg_l1 = regularizers.L1(0.03)
#model.add(layers.Dense(100, activation='relu', kernel_regularizer=reg_l1))

# régularisation sur le biais des neurones
reg_l2 = regularizers.L2(0.01)

# régularsisation sur la sortie de la fonction d'activation
#reg_l1_l2 = regularizers.l1_l2(l1=0.001, l2=0.001)
#model.add(layers.Dense(100, activation='relu', activity_regularizer=reg_l1_l2))
model.add(layers.Dense(1, activation='sigmoid'))
# afficher les paramètres du modèle
model.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 100)            │         1,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,801 (7.04 KB)

 Trainable params: 1,801 (7.04 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#baselearningrate=1
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
 #             optimizer = tf.keras.optimizers.Adam(learning_rate=baselearningrate/10),
  #            metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [8]:
# spécifie loss, optimizer et métrique d'évaluation
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
 #             optimizer = tf.keras.optimizers.Adam(),
  #            metrics=['accuracy'])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adamax(),
              metrics=['accuracy'])



#model.compile(
 #   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  #  optimizer=tf.keras.optimizers.Adamax(learning_rate=0.002),
   # metrics=['accuracy']
#)


## Visualiser courbes avec tenserboard

In [9]:
import datetime

# Définir le chemin absolu pour log_dir
log_dir = r"C:\Users\ContactM2\Documents\DESU_LC\Rochet\DL\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
history = model.fit(X_train, y,
          validation_split=0.3,
          batch_size=16,
          epochs=200,
          callbacks=[tensorboard_callback])

Epoch 1/200


c:\Users\ContactM2\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1799/1799 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4980 - loss: 239.7962

c:\Users\ContactM2\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1799/1799 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.4980 - loss: 239.7322 - val_accuracy: 0.5000 - val_loss: 65.4865
Epoch 2/200
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5085 - loss: 88.4946 - val_accuracy: 0.4995 - val_loss: 93.5664
Epoch 3/200
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5026 - loss: 79.2596 - val_accuracy: 0.4986 - val_loss: 51.7408
Epoch 4/200
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5045 - loss: 77.1745 - val_accuracy: 0.5004 - val_loss: 228.4493
Epoch 5/200
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5107 - loss: 80.2027 - val_accuracy: 0.4998 - val_loss: 118.0872
Epoch 6/200
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5158 - loss: 64.1090 - val_accuracy: 0.5003 - val_loss: 123.4745
Epoch 7/200
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5249 - loss: 52.4501 - val_accuracy: 0.5078 - val_loss: 64.6802
Epoch 8/200
1799/1799 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5195 - l

In [ ]:
## Pour visualiser les courbes, lancer tensorboard depuis le terminal :
# tensorboard --logdir "C:\Users\ContactM2\Documents\DESU_LC\Rochet\DL\logs"
# tensorboard --logdir "C:\Users\ContactM2\Documents\DESU_LC\Rochet\DL\20240713-003002"
# Puis dans le navigateur : http://localhost:6006

In [13]:
# Sauvegarder le modèle
model.save('model_spotify_test.keras')

In [ ]:
# Recharger le modèle
loaded_model = tf.keras.models.load_model('model_spotify_test.keras')

# Vérifier que le modèle a été correctement chargé
loaded_model.summary()

# Optionnel : Réévaluer le modèle sur les données d'entraînement pour vérifier qu'il fonctionne correctement
loaded_model.evaluate(X_train, y_train)